In [ ]:
!pip install praw

In [ ]:
!pip install langdetect

In [ ]:
import pandas as pd
#import praw
#import pprint
import numpy as np
import datetime
import string 

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')

from collections import defaultdict

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


DATA CRAWLING

In [ ]:
reddit = praw.Reddit(client_id='Xgjr11N8PlAQD5LQVFBshA', client_secret='dh2yjq5tJ9laOm-Ma02cohsKm61Ygg', user_agent='Data Crawling')

In [ ]:
subreddit= reddit.subreddit('india')

In [ ]:
keywords = ['recession', 'economic downturn', 'unemployment', 'layoff', 'debt', 'slowdown',
            'financial crisis', 'poverty']


# Define the attributes of the posts you want to extract
attributes = ['created_utc', 'title', 'score','id','selftext', 'author']
c_attributes = ['created_utc','author','score','body']

# Create an empty list to store the posts
posts = []
comment_list = []

In [ ]:
for keyword in keywords:
  for post in subreddit.search(keyword, limit=1000):
      # Extract the attributes from the post
      post_attributes = []
      for attribute in attributes:

          if attribute == 'created_utc':
            utc_timestamp = getattr(post, attribute)
            # Convert the UTC timestamp to a datetime object
            utc_datetime = datetime.datetime.utcfromtimestamp(utc_timestamp)
            # Convert the datetime object to a readable date string
            date_string = utc_datetime.strftime('%Y-%m-%d %H:%M:%S')
            post_attributes.append(date_string)
            
          else:
            post_attributes.append(getattr(post, attribute))
      # Add the attributes to the list of posts
      posts.append(post_attributes)

      #Extract the comments from the post 
      post.comments.replace_more(limit=0)
      for comment in post.comments:
        comment_attributes = []

        utc_timestamp = getattr(post, 'created_utc')
        utc_datetime = datetime.datetime.utcfromtimestamp(utc_timestamp)
        date_string = utc_datetime.strftime('%Y-%m-%d %H:%M:%S')
        comment_attributes.append(date_string)

        comment_attributes.append(getattr(post,'title'))

        for attribute in c_attributes:

            if attribute == 'created_utc':
              utc_timestamp = getattr(comment, attribute)
              # Convert the UTC timestamp to a datetime object
              utc_datetime = datetime.datetime.utcfromtimestamp(utc_timestamp)
              # Convert the datetime object to a readable date string
              date_string = utc_datetime.strftime('%Y-%m-%d %H:%M:%S')
              comment_attributes.append(date_string)
              
            else:
              comment_attributes.append(getattr(comment, attribute))
              
        # Add the attributes to the list of comments
        comment_list.append(comment_attributes)

In [ ]:
# Convert the list of posts to a pandas dataframe
df_post = pd.DataFrame(posts, columns=attributes)
df_post = df_post.sort_values('created_utc')
df_post = df_post.drop_duplicates('title')
df_post

,created_utc,title,score,id,selftext,author


In [ ]:
df_comment = pd.DataFrame(comment_list, columns=["post_date","post_title"] + c_attributes)
df_comment = df_comment.sort_values('created_utc')
df_comment = df_comment.drop_duplicates('body')
df_comment = df_comment.iloc[1151:]

In [ ]:
# Drop any duplicate rows
df_post = df_post.drop_duplicates('title')

# Drop any rows with missing values
df_post = df_post.dropna()

# Remove any URLs from the text
df_post['selftext'] = df_post['selftext'].str.replace(r'http\S+', '', regex=True)

In [ ]:
# Drop any duplicate rows
df_comment = df_comment.drop_duplicates('body')

# Remove any URLs/Bots/Reposting of news articles from the text
df_comment['body'] = df_comment['body'].str.replace(r'[ ]', " ", regex=True)
df_comment = df_comment[df_comment["body"].str.contains("For those blocked by paywall") == False]
df_comment = df_comment[df_comment["body"].str.contains("I am a bot, and this action") == False]
df_comment = df_comment[df_comment["body"].str.contains("https://github.com") == False]

# Drop any rows with missing values
df_comment= df_comment.dropna()

In [ ]:
df.to_csv('post.csv', mode='a', header=False,index=False)
df.to_csv('comment.csv', mode='a', header=False,index=False)